In [1]:
import requests
import pandas as pd
import numpy as np
import folium
import random
import matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors
from IPython.display import Image 
from IPython.core.display import HTML 
from pandas.io.json import json_normalize
!pip install geopy
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans

## Read the previous saved merged csv file from part 2

In [18]:

merged_df = pd.read_csv('Toronto_neighbourhoods_merged.csv')
merged_df.drop(["Unnamed: 0"], axis = 1, inplace = True)
merged_df.head(5)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Create a map for Toronto to explore its neighborhood

In [19]:
address = 'Toronto, ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

C:\Users\ruthv\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.19.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Toronto are 43.653963, -79.387207.


## create Toronto map using previously obtained latitude and longitude values from the geocoder

In [20]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(merged_df['Latitude'], merged_df['Longitude'], merged_df['Borough'], merged_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Removed my credentials for privacy concerns

In [5]:
CLIENT_ID = '************************************************'
CLIENT_SECRET = '*****************************************'
VERSION = '********'
LIMIT = 100

## Create a method to get the near by venues using foursquare API

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
toronto_venues = getNearbyVenues(names = merged_df['Neighborhood'],
                                   latitudes = merged_df['Latitude'],
                                   longitudes = merged_df['Longitude'])

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [23]:
toronto_venues.shape

(2246, 7)

## Group and  Count the no. of neighborhoods

In [24]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",2,2,2,2,2,2
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",9,9,9,9,9,9
"Alderwood,Long Branch",10,10,10,10,10,10
"Bathurst Manor,Downsview North,Wilson Heights",17,17,17,17,17,17
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",25,25,25,25,25,25
Berczy Park,56,56,56,56,56,56


## Let us find the unique categories from the venues

In [25]:
print('{} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

273 unique categories.


In [26]:
## Analyze each venue
toronto_oneenc = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_oneenc['Neighborhood'] = toronto_venues['Neighborhood'] 

fixed_columns = [toronto_oneenc.columns[-1]] + list(toronto_oneenc.columns[:-1])
toronto_oneenc = toronto_oneenc[fixed_columns]

toronto_oneenc.head()


,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
toronto_mean_venues = toronto_oneenc.groupby('Neighborhood').mean().reset_index()
toronto_mean_venues.shape

(101, 273)

In [28]:
num_top_venues = 5

for hood in toronto_mean_venues['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_mean_venues[toronto_mean_venues['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['Venue Type','Frequency']
    temp = temp.iloc[1:]
    temp['Frequency'] = temp['Frequency'].astype(float)
    temp = temp.round({'Frequency': 2})
    print(temp.sort_values('Frequency', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
            Venue Type  Frequency
0          Coffee Shop       0.06
1                 Café       0.04
2           Steakhouse       0.04
3      Thai Restaurant       0.04
4  American Restaurant       0.04


----Agincourt----
       Venue Type  Frequency
0          Lounge       0.25
1  Clothing Store       0.25
2    Skating Rink       0.25
3  Breakfast Spot       0.25
4     Yoga Studio       0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                        Venue Type  Frequency
0                       Playground        0.5
1                             Park        0.5
2                    Metro Station        0.0
3              Monument / Landmark        0.0
4  Molecular Gastronomy Restaurant        0.0


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
       Venue Type  Frequency
0   Grocery Store       0.22
1        Pharmacy       0.11
2  Sandwich Place       0.11
3 

    Venue Type  Frequency
0     Pharmacy       0.11
1  Supermarket       0.11
2       Bakery       0.11
3  Music Venue       0.06
4      Brewery       0.06


----Downsview Central----
            Venue Type  Frequency
0           Food Truck        0.5
1       Baseball Field        0.5
2          Yoga Studio        0.0
3   Mexican Restaurant        0.0
4  Monument / Landmark        0.0


----Downsview Northwest----
             Venue Type  Frequency
0  Gym / Fitness Center        0.2
1         Grocery Store        0.2
2          Liquor Store        0.2
3        Discount Store        0.2
4    Athletics & Sports        0.2


----Downsview West----
      Venue Type  Frequency
0  Moving Target        0.2
1  Shopping Mall        0.2
2  Grocery Store        0.2
3          Hotel        0.2
4           Bank        0.2


----Downsview,North Park,Upwood Park----
                   Venue Type  Frequency
0                        Park       0.25
1  Construction & Landscaping       0.25
2            

----Rouge,Malvern----
                        Venue Type  Frequency
0             Fast Food Restaurant        1.0
1               Mexican Restaurant        0.0
2                            Motel        0.0
3              Monument / Landmark        0.0
4  Molecular Gastronomy Restaurant        0.0


----Runnymede,Swansea----
           Venue Type  Frequency
0         Coffee Shop       0.11
1                Café       0.08
2         Pizza Place       0.08
3  Italian Restaurant       0.05
4    Sushi Restaurant       0.05


----Ryerson,Garden District----
                  Venue Type  Frequency
0                Coffee Shop       0.08
1             Clothing Store       0.07
2                       Café       0.04
3             Cosmetics Shop       0.03
4  Middle Eastern Restaurant       0.03


----Scarborough Village----
            Venue Type  Frequency
0           Playground        0.5
1        Grocery Store        0.5
2   Mexican Restaurant        0.0
3                Motel        0.0
4 

## Create a method to find most common venues

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [30]:
## create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_mean_venues['Neighborhood']
for ind in np.arange(toronto_mean_venues.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_mean_venues.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Thai Restaurant,Steakhouse,Café,American Restaurant,Burger Joint,Bakery,Salad Place,Bar,Asian Restaurant
1,Agincourt,Lounge,Breakfast Spot,Clothing Store,Skating Rink,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Playground,Park,Women's Store,Dog Run,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pizza Place,Fried Chicken Joint,Pharmacy,Sandwich Place,Fast Food Restaurant,Beer Store,Coffee Shop,General Entertainment,Donut Shop
4,"Alderwood,Long Branch",Pizza Place,Pharmacy,Pool,Skating Rink,Sandwich Place,Athletics & Sports,Pub,Coffee Shop,Gym,Colombian Restaurant
5,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Shopping Mall,Diner,Frozen Yogurt Shop,Bank,Bridal Shop,Fast Food Restaurant,Sushi Restaurant,Sandwich Place,Pizza Place
6,Bayview Village,Bank,Japanese Restaurant,Café,Chinese Restaurant,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Women's Store
7,"Bedford Park,Lawrence Manor East",Coffee Shop,Italian Restaurant,Fast Food Restaurant,Juice Bar,Café,Pizza Place,Indian Restaurant,Ice Cream Shop,Pub,Sushi Restaurant
8,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Cheese Shop,Steakhouse,Farmers Market,Seafood Restaurant,Pub,Café,Bakery
9,"Birch Cliff,Cliffside West",General Entertainment,College Stadium,Skating Rink,Café,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store


## Cluster Neighborhoods
## Run k-means to cluster the neighborhood into 5 clusters.

In [31]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_mean_venues.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 4, 0, 0, 0, 0, 0, 0, 0])

In [38]:
# add clustering labels
toronto_merged = merged_df.head(100)
toronto_merged['Cluster Labels'] = pd.DataFrame(kmeans.labels_)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].fillna(0.0).astype(int)
toronto_merged.head()


C:\Users\ruthv\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,0,Fast Food Restaurant,Women's Store,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,0,Bar,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,4,Breakfast Spot,Medical Center,Rental Car Location,Intersection,Mexican Restaurant,Electronics Store,Spa,Pizza Place,Donut Shop,Doner Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Korean Restaurant,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Fried Chicken Joint,Caribbean Restaurant,Hakka Restaurant,Thai Restaurant,Bakery,Athletics & Sports,Bank,Department Store,Dessert Shop,Dim Sum Restaurant


In [39]:
# create final map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine the clusters

In [40]:
## Examine Clusters
## Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0,Fast Food Restaurant,Women's Store,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
1,Scarborough,0,Bar,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
3,Scarborough,0,Coffee Shop,Korean Restaurant,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
4,Scarborough,0,Fried Chicken Joint,Caribbean Restaurant,Hakka Restaurant,Thai Restaurant,Bakery,Athletics & Sports,Bank,Department Store,Dessert Shop,Dim Sum Restaurant
5,Scarborough,0,Grocery Store,Playground,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
6,Scarborough,0,Coffee Shop,Department Store,Discount Store,Convenience Store,Hobby Shop,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Doner Restaurant
7,Scarborough,0,Bakery,Bus Line,Park,Bus Station,Fast Food Restaurant,Metro Station,Intersection,Soccer Field,Construction & Landscaping,Dim Sum Restaurant
8,Scarborough,0,Motel,American Restaurant,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
9,Scarborough,0,General Entertainment,College Stadium,Skating Rink,Café,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
10,Scarborough,0,Indian Restaurant,Pet Store,Vietnamese Restaurant,Latin American Restaurant,Chinese Restaurant,Dog Run,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant


## Most of the cluster 1 venues have restaurants

In [42]:
## Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
81,York,1,Pizza Place,Brewery,Caribbean Restaurant,Bus Line,Grocery Store,Airport,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant,Electronics Store


## the above cluster has restaurants with varied cuisines

In [43]:
## Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,Downtown Toronto,2,Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Boutique,Plane


##  Cluster 3 has most of the airport related services

In [44]:
## Cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,East York,3,Pharmacy,Video Store,Curling Ice,Cosmetics Shop,Athletics & Sports,Asian Restaurant,Skating Rink,Park,Beer Store,Electronics Store
42,East Toronto,3,Pet Store,Pub,Liquor Store,Brewery,Sandwich Place,Fast Food Restaurant,Burger Joint,Italian Restaurant,Steakhouse,Fish & Chips Shop
56,Downtown Toronto,3,Coffee Shop,Cocktail Bar,Restaurant,Cheese Shop,Steakhouse,Farmers Market,Seafood Restaurant,Pub,Café,Bakery


## This cluster has most of the entertainment related venues

In [45]:
## Cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,4,Breakfast Spot,Medical Center,Rental Car Location,Intersection,Mexican Restaurant,Electronics Store,Spa,Pizza Place,Donut Shop,Doner Restaurant
13,Scarborough,4,Pizza Place,Pharmacy,Fried Chicken Joint,Shopping Mall,Noodle House,Chinese Restaurant,Fast Food Restaurant,Italian Restaurant,Thai Restaurant,Dim Sum Restaurant
16,Scarborough,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,East York,4,Indian Restaurant,Yoga Studio,Park,Pharmacy,Pizza Place,Burger Joint,Sandwich Place,Liquor Store,Bank,Supermarket
41,East Toronto,4,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Café,Caribbean Restaurant,Diner,Brewery,Indian Restaurant
59,Downtown Toronto,4,Coffee Shop,Aquarium,Hotel,Café,Italian Restaurant,Pizza Place,Brewery,Bakery,Scenic Lookout,Fried Chicken Joint
63,Central Toronto,4,Garden,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant,Dance Studio
73,York,4,Trail,Field,Dog Run,Hockey Arena,Women's Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
75,Downtown Toronto,4,Grocery Store,Café,Park,Convenience Store,Nightclub,Athletics & Sports,Italian Restaurant,Restaurant,Diner,Baby Store
90,Etobicoke,4,Park,River,Smoke Shop,Women's Store,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner


## The last cluster has parks, gardens and restaurants signifying the venues near to an urban location